In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import natsort, random, copy
import os, glob, sys
import cv2

import time, timeit
from datetime import datetime

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split

In [4]:
# 환경변수 설정
sys.path.append("..")
module_names=['SwinUNet']
for module_name in module_names:
    exec('from models.'+module_name+' import *')
model_names= ['SwinUNet']

In [5]:
train_size = 0.7
in_channels = 3

number_of_classes = 1
loss_function = 'Tversky Focal Loss'
Optimizers = ['AdamW']
LRs = [1e-3]
batch_size = 8
epochs = 50
iterations = [1,10]
devices = [0,3] 
Target_Datasets = ['reflex_mask']
Dataset_dir = 'Dataset'

In [6]:
def control_random_seed(seed, pytorch=True):
    random.seed(seed)
    np.random.seed(seed)
    try:
        torch.manual_seed(seed)
        if torch.cuda.is_available()==True:
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except:
        pass
        torch.backends.cudnn.benchmark = False

In [7]:
def imread_kor ( filePath, mode=cv2.IMREAD_UNCHANGED ) : 
    stream = open( filePath.encode("utf-8") , "rb") 
    bytes = bytearray(stream.read()) 
    numpyArray = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(numpyArray , mode)

In [8]:
def imwrite_kor(filename, img, params=None): 
    try: 
        ext = os.path.splitext(filename)[1] 
        result, n = cv2.imencode(ext, img, params) 
        if result:
            with open(filename, mode='w+b') as f: 
                n.tofile(f) 
                return True
        else: 
            return False 
    except Exception as e: 
        print(e) 
        return False

In [9]:
class ImagesDataset(Dataset):
    def __init__(self, image_path_list, target_path_list, transform=None):
        self.image_path_list = image_path_list
        self.target_path_list = target_path_list
        self.transform = transform

    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        image_path = self.image_path_list[idx]
        mask_path = self.target_path_list[idx]
        image = np.load(image_path) 
        mask = imread_kor(mask_path)
        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)
        mask[mask > 0] = 1
        return image, mask, image_path

In [10]:
def adjust_learning_rate(optimizer, epoch, lr):
    lr = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
### losses & accuracy ### https://github.com/Jo-dsa/SemanticSeg/blob/master/src/utils.py
def tversky_index(yhat, ytrue, alpha=0.3, beta=0.7, epsilon=1e-6):
    """
    Computes Tversky index
    Args:
        yhat (Tensor): predicted masks
        ytrue (Tensor): targets masks
        alpha (Float): weight for False positive
        beta (Float): weight for False negative
                    `` alpha and beta control the magnitude of penalties and should sum to 1``
        epsilon (Float): smoothing value to avoid division by 0
    output:
        tversky index value
    """
    TP = torch.sum(yhat * ytrue, (1,2,3))
    FP = torch.sum((1. - ytrue) * yhat, (1,2,3))
    FN = torch.sum((1. - yhat) * ytrue, (1,2,3))
    
    return TP/(TP + alpha * FP + beta * FN + epsilon)

In [12]:
def tversky_focal_loss(yhat, ytrue, alpha=0.7, beta=0.3, gamma=0.75):
    """
    Computes tversky focal loss for highly umbalanced data
    https://arxiv.org/pdf/1810.07842.pdf
    Args:
        yhat (Tensor): predicted masks
        ytrue (Tensor): targets masks
        alpha (Float): weight for False positive
        beta (Float): weight for False negative
                    `` alpha and beta control the magnitude of penalties and should sum to 1``
        gamma (Float): focal parameter
                    ``control the balance between easy background and hard ROI training examples``
    output:
        tversky focal loss value with `mean` reduction
    """
    return torch.mean(torch.pow(1 - tversky_index(yhat, ytrue, alpha, beta), gamma))

In [13]:
def focal_loss(yhat, ytrue, alpha=0.75, gamma=2):
    """
    Computes α-balanced focal loss from FAIR
    https://arxiv.org/pdf/1708.02002v2.pdf
    Args:
        yhat (Tensor): predicted masks
        ytrue (Tensor): targets masks
        alpha (Float): weight to balance Cross entropy value
        gamma (Float): focal parameter
    output:
        loss value with `mean` reduction
    """

    # compute the actual focal loss
    focal = -alpha * torch.pow(1. - yhat, gamma) * torch.log(yhat)
    f_loss = torch.sum(ytrue * focal, dim=1)

    return torch.mean(f_loss)

In [14]:
def Intersection_over_Union(yhat, ytrue, threshold=0.5, epsilon=1e-6, nan_process = 'remove'):
    """
    Computes Intersection over Union metric
    Args:
        yhat (Tensor): predicted masks (batch_size, 1, height, width)
        ytrue (Tensor): targets masks (batch_size, 1, height, width)
        threshold (Float): threshold for pixel classification
        epsilon (Float): smoothing parameter for numerical stability
    output:
        iou value with `mean` reduction
    """
    intersection = ((yhat>threshold).long() & ytrue.long()).float().sum((1,2,3))
    union = ((yhat>threshold).long() | ytrue.long()).float().sum((1,2,3))
    if nan_process == 'remove': # if sum of true == 0, remove
        sum_bool = torch.sum(torch.flatten(ytrue,1),1).bool()
        iou =(intersection/(union))#.reshape(intersection.shape[0],-1)
        iou = torch.nanmean(iou,dim=0)
        if torch.isnan(iou):
            return 0
        return (torch.mean(iou)).item()

In [15]:
def Dice_Coefficient(yhat, ytrue, epsilon=1e-6, nan_process = 'remove'):
    """
    Computes a soft Dice Loss
    Args:
        yhat (Tensor): predicted masks (batch_size, 1, height, width)
        ytrue (Tensor): targets masks (batch_size, 1, height, width)
        epsilon (Float): smoothing value to avoid division by 0
    output:
        DL value with `mean` reduction
    """
    # compute Dice components
    intersection = torch.sum(yhat * ytrue, (1,2,3))
    cardinal = torch.sum(yhat + ytrue, (1,2,3))
    if nan_process == 'remove': # if sum of true == 0, remove
        sum_bool = torch.sum(torch.flatten(ytrue,1),1).bool()
        dice = (2 * intersection / (cardinal))#.reshape(intersection.shape[0],-1)
        dice = torch.nanmean(dice,dim=0)
        if torch.isnan(dice):
            return 0
        return (torch.mean(dice)).item()

In [16]:
def train(train_loader, epoch, \
          model, criterion, optimizer, device
          ):
    model.train()
    limit=0
    train_losses=AverageMeter()
    for i, (input, target, _) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        ss = model(input)
        output = nn.ReLU()(model(input))
#         print('input: ', input.size())
#         print('target: ', target.size())
#         print('ss: ', ss.size())
#         print('output: ', output.size())
        loss = criterion(output,target).float()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        train_losses.update(loss.detach().cpu().numpy(),input.shape[0])
    Train_Loss=np.round(train_losses.avg,6)
    return Train_Loss

In [17]:
def train(train_loader, epoch, \
          model, criterion, optimizer, device
          ):
    model.train()
    limit=0
    train_losses=AverageMeter()
    for i, (input, target, _) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        ss = model(input)
        output = nn.Sigmoid()(model(input))
#         print('input: ', input.size())
#         print('target: ', target.size())
#         print('ss: ', ss.size())
#         print('output: ', output.size())
        loss = criterion(output,target).float()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        train_losses.update(loss.detach().cpu().numpy(),input.shape[0])
    Train_Loss=np.round(train_losses.avg,6)
    return Train_Loss

In [18]:
def validate(validation_loader, 
          model, criterion, device,
        model_path=False,
             return_image_paths=False,
          ):
    if model_path!=False:
        model.load_state_dict(torch.load(model_path))
    model.eval()
    for i, (input, target, image_path) in enumerate(validation_loader):
        input =input.to(device)
        target = target.to(device)
        with torch.no_grad():
            output = nn.Sigmoid()(model(input))
#         print('output : ', output)
        if i==0:
#             print('i = 0')
            targets=target
            outputs=output
            if return_image_paths==True:
                image_paths = image_path
        else:
#             print('i  != 0')
            targets=torch.cat((targets,target))
            outputs=torch.cat((outputs,output),axis=0)
            if return_image_paths==True:
                image_paths += image_path
    if return_image_paths==True:
        return outputs, targets, image_paths
    return outputs, targets

In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [20]:
def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

In [21]:
def Do_Experiment(iteration, model_name, model, Optimizer, lr,  number_of_classes, epochs, Metrics,df,device):
    train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, transform)
    validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list, transform)
    test_dataset = ImagesDataset(test_image_path_list, test_target_path_list, transform)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size,
    num_workers=num_workers, pin_memory=pin_memory
    )
    validation_loader = torch.utils.data.DataLoader(
        validation_dataset, batch_size=batch_size, 
        num_workers=num_workers, pin_memory=pin_memory
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, 
        num_workers=num_workers, pin_memory=pin_memory
    )
    start = timeit.default_timer()
    train_bool=True
    test_bool=True
    if loss_function == 'Tversky Focal Loss':
        criterion=tversky_focal_loss
    if Optimizer=='Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif Optimizer == 'SGD':
        momentum = 0.9
        weight_decay = 1e-4
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay=weight_decay)
    elif Optimizer =='AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    save_dir='saved_model'
    try:
        os.mkdir(save_dir)
    except:
        pass
    if train_bool:
        now = datetime.now()
        Train_date=now.strftime("%y%m%d_%H%M%S")
        print('Training Start Time:',Train_date)
        best=9999
        best_epoch=1
        Early_Stop=0
        Early_Stop_Start=30
        train_start_time = timeit.default_timer()
        Train_Losses=[]
        Validation_Losses=[]
        for epoch in range(1, epochs+1):
            adjust_learning_rate(optimizer, epoch, lr)
            Train_Loss = train(train_loader, epoch, 
              model, criterion, optimizer, device
              )
            outputs, targets = validate(validation_loader, 
              model, criterion, device
              )
            Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)            
            iou = np.round(Intersection_over_Union(outputs, targets),3)
            dice = np.round(Dice_Coefficient(outputs, targets),3)
            now = datetime.now()
            date=now.strftime("%y%m%d_%H%M%S")
            print(str(epoch)+'EP('+date+'):',end=' ')
            print('T_Loss: ' + str(Train_Loss), end=' ')
            print('V_Loss: ' + str(Loss), end=' ')
            print('IoU: ' + str(iou), end=' ')
            print('Dice: ' + str(dice), end='\n')
                        
            if Loss<best:
                torch.save(model.state_dict(), save_dir+'/'+model_name+'_'+Dataset_name+'.pt')
                best_epoch = epoch
                best = Loss
                print('Best Epoch:',best_epoch,'Loss:',Loss)
        train_stop_time = timeit.default_timer()
    if test_bool:
        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Test Start Time:',date)
        outputs, targets, image_paths \
            = validate(test_loader, 
              model, criterion, device,
            model_path=save_dir+'/'+model_name+'_'+Dataset_name+'.pt',
                       return_image_paths=True
              )        
        Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)
        iou = np.round(Intersection_over_Union(outputs, targets),3)
        dice = np.round(Dice_Coefficient(outputs, targets),3)

        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Best Epoch:',best_epoch)
        print('Test('+date+'): '+'Loss: ' + str(Loss),end=' ')
        print('IoU: ' + str(iou), end=' ')
        print('Dice: ' + str(dice), end='\n')                    
                            
        stop = timeit.default_timer()
        m, s = divmod((train_stop_time - train_start_time)/epoch, 60)
        h, m = divmod(m, 60)
        Time_per_Epoch = "%02d:%02d:%02d" % (h, m, s)

        m, s = divmod(stop - start, 60)
        h, m = divmod(m, 60)
        Time = "%02d:%02d:%02d" % (h, m, s)
        print(Time)
        
        total_params = sum(p.numel() for p in model.parameters())
        total_params = format(total_params , ',')
        Performances = [iteration, Dataset_name, model_name, loss_function, lr, batch_size, epochs,   Loss, iou, dice, total_params,Time, best_epoch, Time_per_Epoch]
        Performances = pd.Series(Performances, index=df.columns)
        df = pd.concat([df, Performances.to_frame().T], ignore_index=True)
    now = datetime.now()
    date=now.strftime("%y%m%d_%H%M%S")
    print('End',date)
    
    return df

In [22]:
now = datetime.now()
Experiment_date=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiment_date)

Experiment Start Time: 230717_202006


In [23]:
Metrics=['Iteration','Dataset','Model Name', 'Loss Function', 'LR', 'Batch size', '#Epochs',  'Loss', 'mIoU', 'mDice','Total Params','Train-Predction Time','Best Epoch','Time per Epoch']
image_paths_dirs=[]
target_dirs = [path  for path in natsort.natsorted(glob.glob(Dataset_dir+'/Masks/*'))]
for target_dir in target_dirs:
    image_paths_dirs.append(Dataset_dir+'/Originals/'+os.path.basename(target_dir).split('_')[0]+'_ori')
df = pd.DataFrame(index=None, columns=Metrics)
csv_file = False 
try:
    if csv_file != False:
        csv_file_for_modifying = 'Segmentation_Model_Comparison_Performance_220511_045617.csv'
        df= pd.read_csv(csv_file_for_modifying, encoding='cp949')
        df.to_csv('Segmentation_Model_Comparison_Performance_'+Experiment_date+'.csv', index=False, header=True, encoding="cp949")
except:
    df = pd.DataFrame(index=None, columns=Metrics)
print('Batch Size:',batch_size)
print('Train Size',train_size)

Batch Size: 8
Train Size 0.7


In [ ]:
Dataset_Number=0
for image_paths_dir, target_dir in zip(image_paths_dirs, target_dirs):
    Dataset_name = os.path.basename(target_dir)
    if Dataset_name not in Target_Datasets:
        continue
    Dataset_Number+=1
    print(str(Dataset_Number)+'/'+str(len(Target_Datasets)), Dataset_name)
    for iteration in range(iterations[0], iterations[1]+1):
        seed=iteration    
        for model_name in model_names:
            if len(df[(df['Dataset'] ==Dataset_name) & (df['Model Name']== model_name) & (df['Iteration']== iteration)])>0:
                continue
            image_path_list=natsort.natsorted(glob.glob(image_paths_dir+'/*'))
            target_path_list=[]
            for image_path in image_path_list:
                #target_path_list.append(target_dir+'/'+os.path.basename(image_path).replace('npy','tif'))
                target_path_list.append(target_dir+'/'+os.path.basename(image_path).replace('npy','png'))
            transform = transforms.Compose([
                        transforms.ToTensor(),
                ])
            num_workers=4
            shuffle=True
            pin_memory=True
            num_dataset = len(target_path_list)
            indices = list(range(num_dataset))
            split1=int(train_size*num_dataset)
            split2=int((train_size+(1-train_size)/2)*num_dataset)
            control_random_seed(seed)
            if shuffle:
                np.random.shuffle(indices)
            train_idx, validation_idx, test_idx = indices[:split1], indices[split1:split2], indices[split2:]
            train_image_path_list=[]
            train_target_path_list=[]
            validation_image_path_list=[]
            validation_target_path_list=[]
            test_image_path_list=[]
            test_target_path_list=[]
            for i, index in enumerate(indices):
                if i<split1:
                    train_image_path_list.append(image_path_list[index])
                    train_target_path_list.append(target_path_list[index])
                elif split1<=i and i<split2:
                    validation_image_path_list.append(image_path_list[index])
                    validation_target_path_list.append(target_path_list[index])
                else:
                    test_image_path_list.append(image_path_list[index])
                    test_target_path_list.append(target_path_list[index])

            print(model_name)
            for Optimizer in Optimizers :
                for lr in LRs:
                    print('LR:',lr)
                    control_random_seed(seed)
                    in_channels=3
                    model=str_to_class(model_name)(in_channels, number_of_classes)
#                     print('* model load success')
#                     print(model)
                    device = torch.device("cuda:"+str(devices[0]))
                    if len(devices)>1:
                        model = torch.nn.DataParallel(model, device_ids = devices ).to(device)
                    else:
                        model = model.to(device)
                    df = Do_Experiment(iteration, model_name, model, Optimizer, lr,  number_of_classes, epochs, Metrics,df,device)
                    try:
                        df.to_csv('Segmentation_Model_Comparison_Performance_'+Experiment_date+'.csv', index=False, header=True, encoding="cp949")
                    except:
                        now = datetime.now()
                        tmp_date=now.strftime("%y%m%d_%H%M%S")
                        df.to_csv('Segmentation_Model_Comparison_Performance_'+Experiment_date+'_'+tmp_date+'_tmp'+'.csv', index=False, header=True, encoding="cp949")
print('End')
# os._exit(0)

1/1 reflex_mask
SwinUNet
LR: 0.001


/userHome/userhome2/taekyung/miniconda3/envs/swin/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343962757/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Training Start Time: 230717_202008
1EP(230717_202035): T_Loss: 0.883739 V_Loss: 0.621754 IoU: 0.377 Dice: 0.502
Best Epoch: 1 Loss: 0.621754
2EP(230717_202058): T_Loss: 0.515631 V_Loss: 0.530357 IoU: 0.449 Dice: 0.591
Best Epoch: 2 Loss: 0.530357
3EP(230717_202121): T_Loss: 0.515094 V_Loss: 0.408654 IoU: 0.515 Dice: 0.656
Best Epoch: 3 Loss: 0.408654
4EP(230717_202145): T_Loss: 0.47457 V_Loss: 0.404955 IoU: 0.52 Dice: 0.665
Best Epoch: 4 Loss: 0.404955
5EP(230717_202212): T_Loss: 0.434913 V_Loss: 0.413397 IoU: 0.533 Dice: 0.681


In [ ]:
df

ws: 98